In [19]:
import pandas as pd
import numpy as np
import os
import random
import matplotlib.image as mpimg
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import PIL
import torch
from torch import nn
from sklearn.feature_extraction.image import extract_patches_2d
!pip install patchify
from patchify import patchify
from sklearn.model_selection import train_test_split
from google.colab import drive
from PIL import Image, ImageOps
from tqdm import tqdm
from collections import Counter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
drive.mount('/content/drive')

path = r'/content/drive/MyDrive/dissertation_seg_data'

input_path = os.path.join(path,"input_resize_patches")

mask_path = os.path.join(path,"mask_resize_patches")

train_split = 0.8

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
image_input_path = sorted([ os.path.join(input_path, fname) for fname in os.listdir(input_path) if fname.endswith(".png") and (('chicago') in fname or 'potsdam' in fname) ])

image_input_path = image_input_path[:round(len(image_input_path)/4)]

len(image_input_path)

740

In [22]:
image_mask_path = sorted([ os.path.join(mask_path, fname) for fname in os.listdir(mask_path) if fname.endswith(".png") and (('chicago') in fname or 'potsdam' in fname)])

image_mask_path = image_mask_path[:round(len(image_mask_path)/4)]
                                  
len(image_mask_path)

740

In [23]:
img_dimensions = []

for image in tqdm(range(len(image_mask_path))):
    inp_path = image_mask_path[image]
    img = Image.open(inp_path)
    img_dimensions.append(img.size)

counter_object = Counter(img_dimensions)

pd.DataFrame.from_dict(counter_object, orient='index').reset_index()

100%|██████████| 740/740 [00:04<00:00, 153.41it/s]


,index,0
0,"(512, 512)",740


In [63]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img

class dataset_loader(keras.utils.Sequence):

    def __init__(self, batch_size, img_size, image_input_path, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.image_input_path = image_input_path
        self.image_mask_path = image_mask_path

    def __len__(self):
        return len(self.image_mask_path) // self.batch_size

    def __getitem__(self, idx):
        i = idx * self.batch_size
        batch_input_img_paths = self.image_input_path[i : i + self.batch_size]
        batch_target_img_paths = self.image_mask_path[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=self.img_size)
            x[j] = img
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")
            y[j] = np.expand_dims(img, 2)
        return x, y

In [64]:
import random

patch_size = (512,512)

batch_size = 64

val_samples = 1000

random.Random(60).shuffle(image_input_path)

random.Random(60).shuffle(image_mask_path)

train_input_img_paths = image_input_path[:-val_samples]

train_target_img_paths = image_mask_path[:-val_samples]

val_input_img_paths = image_input_path[-val_samples:]

val_target_img_paths = image_mask_path[-val_samples:]

train_gen = dataset_loader(batch_size, patch_size, train_input_img_paths, train_target_img_paths)

val_gen = dataset_loader(batch_size, patch_size, val_input_img_paths, val_target_img_paths)


In [72]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, GaussianNoise
from keras.layers import Conv2D, MaxPool2D, Input, concatenate, Conv2DTranspose
from keras.layers.convolutional import MaxPooling2D
from keras import regularizers, optimizers
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.applications import VGG16, InceptionV3
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [66]:
def unet(img_shape):
    # input layer shape is equal to patch image size
    inputs = Input(shape=img_shape)

    previous_block_activation = inputs  # Set aside residual

    contraction = {}
    # # Contraction path: Blocks 1 through 5 are identical apart from the feature depth
    for f in [32, 64, 128, 256]:
        x = Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(previous_block_activation)
        x = Dropout(0.1)(x)
        x = Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        contraction[f'conv{f}'] = x
        x = MaxPooling2D((2, 2))(x)
        previous_block_activation = x

    c5 = Conv2D(160, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(previous_block_activation)
    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(160, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    previous_block_activation = c5

    # Expansive path: Second half of the network: upsampling inputs
    for f in reversed([32, 64, 128, 256]):
        x = Conv2DTranspose(f, (2, 2), strides=(2, 2), padding='same')(previous_block_activation)
        x = concatenate([x, contraction[f'conv{f}']])
        x = Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        x = Dropout(0.2)(x)
        x = Conv2D(f, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
        previous_block_activation = x

    outputs = Conv2D(filters=4, kernel_size=(1, 1), activation="softmax")(previous_block_activation)

    return Model(inputs=inputs, outputs=outputs)

# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()
     

In [67]:
model = unet(img_shape=(512, 512, 3))

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 dropout (Dropout)              (None, 512, 512, 32  0           ['conv2d[0][0]']                 
                                )                                                             

In [68]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"],run_eagerly=True)

In [69]:
import tensorflow as tf

print(tf.config.run_functions_eagerly(True))

None


In [75]:
model.fit(train_gen, epochs=20, validation_data=val_gen, verbose=1)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


ResourceExhaustedError: ignored